# Get the final dataset

**Goal:** this notebook computes the final dataset used in the next part from the raw impact. It combines a lot of knowledge from the [`/analysis/description/first_study`](https://github.com/ElsaB/impact-annotator/tree/master/analysis/description/first_study) folder to filter, clean, curate and annotate the raw dataset. The final annotation kept is the one from VEP. All the operations made are stored in the [`compute_final_dataset.R`](https://github.com/ElsaB/impact-annotator/blob/master/data/utils/compute_final_dataset.R) file, and can be applied on the raw dataset by using the `get_final_dataset()` function.

This notebook is divided in 3 parts:
* **1. Get click_annotvcf annotations**
* **2. Adapt the `get_cleaned_impact()` function**
* **3. Process raw features**

In [2]:
source("../../src/utils/custom_tools.R")
setup_environment("../../src/utils")

Warning message:
“package ‘dplyr’ was built under R version 3.5.1”

In [3]:
impact <- read.table("../../data/all_IMPACT_mutations_180508.txt", sep = "\t", stringsAsFactors = FALSE, header = TRUE)

In [ ]:
nrow(impact)

## Get click_annotvcf annotations

Get the features from `click_annotvcf_IMPACT_mutations_180508.txt` (impact annotated by the click_annotvcf pipeline).

### Get the raw data

In [ ]:
impact_annotated <- read.table("../../../data/annotate_with_click_annotvcf/click_annotvcf_IMPACT_mutations_180508.txt",
                               sep = "\t", stringsAsFactors = FALSE, header = TRUE, comment = "#")

In [ ]:
ncol(impact_annotated)
nrow(impact_annotated)
head(impact_annotated)

We keep the following features:
* `id_colnames`: variant identification columns (chromosome, start position, ...)
* `vag_colnames`: VAGrENT most deleterious annotation
* `vep_colnames`: VEP v92 annotations
* `vep_add_colnames`: VEP v92 annotations (polyphen: 2.2.2, sift: sift5.2.2, COSMIC v81)
    * `VEP_IMPACT`: Subjective impact classification of consequence type
    * `VEP_CLIN_SIG`: ClinVar clinical significance of the dbSNP variant
    * `VEP_Existing_variation`: Identifier(s) of co-located known variants
    * `VEP_COSMIC_CNT`: How many samples have this mutation
    * ...
* `vep_gnomad_colnames`: VEP v92 annotations (annotated with `/ifs/work/leukgen/home/leukbot/tests/vep/gnomad_genomes/gnomad.genomes.r2.0.1.sites.noVEP.vcf.gz` and `/ifs/work/leukgen/home/leukbot/tests/vep/gnomad_exomes/gnomad.exomes.r2.0.1.sites.noVEP.vcf.gz`)
    * `VEP_gnomAD_AF`: Frequency of existing variant in gnomAD exomes combined population (VEP only annotation)
    * `VEP_gnomAD_genome_AC.AN_<POP>`: Allele count | Total number of alleles among `<POP>` genomes, `<POP>` being one of:
        * `AFR`: African/African American
        * `AMR`: Admixed American
        * `ASJ`: Ashkenazi Jewish
        * `EAS`: East Asian
        * `FIN`: Finnish
        * `NFE`: Non-Finnish European
        * `OTH`: Other (population not assigned)
    * `VEP_gnomAD_exome_AC.AN_<POP>`: Allele count | Total number of alleles among `<POP>` exomes

In [ ]:
id_colnames  <- c("ID_VARIANT",
                  "CHR",
                  "START",
                  "END",
                  "REF",
                  "ALT")

vag_colnames <- c("VAG_VT",
                  "VAG_GENE",
                  "VAG_cDNA_CHANGE",
                  "VAG_PROTEIN_CHANGE",
                  "VAG_EFFECT")

vep_colnames <- c("VEP_Consequence",
                  "VEP_SYMBOL",
                  "VEP_HGVSc",
                  "VEP_HGVSp",
                  "VEP_Amino_acids",
                  "VEP_VARIANT_CLASS",
                  "VEP_EXON",
                  "VEP_INTRON")

vep_add_colnames <- c("VEP_IMPACT",
                      "VEP_CLIN_SIG",
                      "VEP_SIFT",
                      "VEP_PolyPhen",
                      "VEP_Existing_variation",
                      "VEP_COSMIC_CNT")

vep_gnomad_colnames <- c("VEP_gnomAD_AF",

                         "VEP_gnomAD_genome_AC.AN_AFR",
                         "VEP_gnomAD_genome_AC.AN_AMR",
                         "VEP_gnomAD_genome_AC.AN_ASJ",
                         "VEP_gnomAD_genome_AC.AN_EAS",
                         "VEP_gnomAD_genome_AC.AN_FIN",
                         "VEP_gnomAD_genome_AC.AN_NFE",
                         "VEP_gnomAD_genome_AC.AN_OTH",

                         "VEP_gnomAD_exome_AC.AN_AFR",
                         "VEP_gnomAD_exome_AC.AN_AMR",
                         "VEP_gnomAD_exome_AC.AN_ASJ",
                         "VEP_gnomAD_exome_AC.AN_EAS",
                         "VEP_gnomAD_exome_AC.AN_FIN",
                         "VEP_gnomAD_exome_AC.AN_NFE",
                         "VEP_gnomAD_exome_AC.AN_OTH")

colnames_to_keep <- c(id_colnames, vag_colnames, vep_colnames, vep_add_colnames, vep_gnomad_colnames)

impact_annotated <- impact_annotated[, colnames_to_keep]

### Add the `OLD_REF`, `OLD_ALT` and `OLD_POS` features from the `.vcf`

During the conversion to the `.vcf` (necessary to annotate with click_annotvcf), we modified the `REF`, `ALT` and `POS` features. The old version have been stored in the `INFO` column of the `.vcf`. We parse this column and add the three features `OLD_REF`, `OLD_ALT` and `OLD_POS` to `impact_annotated`.

In [ ]:
impact_vcf <- read.table("../../../data/annotate_with_click_annotvcf/all_IMPACT_mutations_180508.vcf",
                               sep = "\t", stringsAsFactors = FALSE, header = FALSE, comment = "#")
colnames(impact_vcf) <- c("CHROM", "POS", "ID", "REF", "ALT", "QUAL", "FILTER", "INFO", "FORMAT")

In [ ]:
ncol(impact_vcf)
nrow(impact_vcf)
head(impact_vcf)

In [ ]:
dd <- head(unique(impact_vcf$INFO), 10)
dd <- data.frame(dd,
                 sapply(dd, function(x) strsplit(strsplit(x, '=')[[1]][2], '/')[[1]][1]),
                 sapply(dd, function(x) strsplit(strsplit(x, '=')[[1]][2], '/')[[1]][2]),
                 sapply(dd, function(x) strsplit(strsplit(x, '=')[[1]][2], '/')[[1]][3]),
                 row.names = NULL)
colnames(dd) <- c("old", "new1", "new2", "new_3")
dd

In [ ]:
impact_vcf$OLD_REF <- sapply(impact_vcf$INFO, function(x) strsplit(strsplit(x, '=')[[1]][2], '/')[[1]][1])
impact_vcf$OLD_ALT <- sapply(impact_vcf$INFO, function(x) strsplit(strsplit(x, '=')[[1]][2], '/')[[1]][2])
impact_vcf$OLD_POS <- sapply(impact_vcf$INFO, function(x) strsplit(strsplit(x, '=')[[1]][2], '/')[[1]][3])
head(impact_vcf)

In [ ]:
impact_vcf$join_key <- paste(impact_vcf$CHROM, impact_vcf$POS, impact_vcf$REF, impact_vcf$ALT, sep = '_')
head(impact_vcf$join_key)

**Verification 1** The features are not unique for each `join_key`:

In [ ]:
impact_vcf <- unique(impact_vcf)
nrow(dd <- impact_vcf %>% group_by(join_key) %>% filter(n() > 1))

In [ ]:
head(dd[order(dd$join_key),])

63 mutations were not consistently annotated in impact (when considering the `REF`, `ALT` and `POS` features), these lines will be duplicated in `impact_annotated` to take these mutations into account.

In [ ]:
# example of inconsistently annotated mutation that is duplicated in `impact_annotated`
impact_annotated[impact_annotated$ID_VARIANT == "1_2494203_CAGG_C",]

**Verification 2** Make sure every `impact_annotated` `ID_VARIANT` has an analoguous `join_key` in `impact_vcf`:

In [ ]:
impact_annotated <- unique(impact_annotated)
nrow(impact_annotated)

In [ ]:
table(impact_annotated$ID_VARIANT %in% impact_vcf$join_key)

In [ ]:
impact_annotated <- left_join(impact_annotated, impact_vcf[, c("join_key", "OLD_REF", "OLD_ALT", "OLD_POS")], by = c("ID_VARIANT" = "join_key"))

In [ ]:
head(impact_annotated)

As seen before, 63 new duplicated lines for the inconsistently annotated mutations.

In [ ]:
nrow(impact_annotated)
# previous example of inconsistently annotated mutation that is duplicated in `impact_annotated`, but where we managed to get the two different keys to link with impact
impact_annotated[impact_annotated$ID_VARIANT == "1_2494203_CAGG_C",]

### Create keys to join the two dataframes and extract the features

We are going to identify each mutation with a key in both dataframes, allowing us to link each mutation from `impact` to its corresponding mutation in `impact_annotated`. The keys will be: 
* `mut_key` for `impact`
* `CHR`\_`OLD_POS`\_`OLD_REF`\_`OLD_ALT` for `impact_annotated`

In [ ]:
impact_annotated$join_key <- paste(impact_annotated$CHR, impact_annotated$OLD_POS, impact_annotated$OLD_REF, impact_annotated$OLD_ALT, sep = '_')
head(impact_annotated$join_key)
impact$mut_key <- paste(impact$Chromosome,
                        impact$Start_Position,
                        impact$Reference_Allele,
                        impact$Tumor_Seq_Allele2,
                        sep = '_')
head(impact$mut_key)

**Verification 1** Make sure every impact `mut_key` has an analoguous `join_key` in `impact_annotated`:

In [ ]:
table(impact$mut_key %in% impact_annotated$join_key)

**Verification 2** The features are unique for each `join_key`:

In [ ]:
impact_annotated <- unique(impact_annotated)
nrow(impact_annotated %>% group_by(join_key) %>% filter(n() > 1))

In [ ]:
colnames_to_keep <- c(vag_colnames, vep_colnames, vep_add_colnames, vep_gnomad_colnames)

In [ ]:
impact <- left_join(impact, impact_annotated[,c("join_key", colnames_to_keep)], by = c("mut_key" = "join_key"))

In [ ]:
head(impact)

## Adapt the `get_cleaned_impact()` function

### [Remove] the unique-value and redundant features

In [ ]:
# [-7 features] remove the unique-value features
impact[, c("Entrez_Gene_Id",
           "Center",
           "NCBI_Build",
           "Strand",
           "dbSNP_RS",
           "Matched_Norm_Sample_Barcode",
           "variant_status")] <- list(NULL)

In [ ]:
# [-3 features] remove the redundant features
impact[, c("Match_Norm_Seq_Allele1", "Match_Norm_Seq_Allele2", "Tumor_Seq_Allele1")] <- list(NULL)

### [Modify & Remove] keep the coding and splicing `VEP_Consequence` mutations

The `VEP_Consequence` has been calculated for the most deleterious transcript (cf. click_annotvcf pipeline). Then, we have for each mutation a list of possible consequence **for the selected transcript** (so the ;ost deleterious). This list of mutation is ordered by the most deleterious consequence first, it's the one we keep.

In [ ]:
dd <- tail(unique(impact$VEP_Consequence), 10)
dd <- data.frame(dd, sapply(dd, function(x) strsplit(x, '&')[[1]][1]), row.names = NULL)
colnames(dd) <- c("old", "new")
dd

In [ ]:
# [~ every rows] select only the most deleterious VEP consequence
impact$VEP_Consequence <- sapply(impact$VEP_Consequence, function(x) strsplit(x, '&')[[1]][1])

In [ ]:
plot_histogram(impact, "VEP_Consequence", height = 4, flip = TRUE)

In [ ]:
# [-375,418 rows] keep only the coding and splicing VEP_Consequence mutations
impact <- impact[impact$VEP_Consequence %in% c("missense_variant",
                                               "frameshift_variant",
                                               "stop_gained",
                                               "splice_acceptor_variant",
                                               "inframe_deletion",
                                               "splice_donor_variant",
                                               "inframe_insertion",
                                               "start_lost",
                                               "stop_lost"),]

In [ ]:
plot_histogram(impact, "VEP_Consequence", height = 2, flip = TRUE)

In [ ]:
nrow(impact) # it was 588,547 before

### [Remove] `confidence_class = UNKNOWN` or `confidence_class = OK_NOT_SO`

In [ ]:
get_table(impact$confidence_class)

In [ ]:
get_table(impact$Consequence[impact$confidence_class == "UNKNOWN"])
get_table(impact$VEP_Consequence[impact$confidence_class == "UNKNOWN"])

In [ ]:
# [-5,496 rows] remove rows having `confidence_class = UNKNOWN` or `confidence_class = OK_NOT_SO`
impact <- impact[! impact$confidence_class %in% c("UNKNOWN", "OK_NOT_SO"),]

### [Remove] `minor_contamination` > 0.01

In [ ]:
print_count_and_proportion(nrow(impact[impact$minor_contamination > 0.01,]), nrow(impact))

In [ ]:
# [-9,156 rows] remove the contaminated rows minor_contamination > 0.01
impact <- impact[impact$minor_contamination <= 0.01,]
# [-1 feature] remove the minor_contamination feature
impact["minor_contamination"] <- NULL

### [Remove] `n_depth` < 20

In [ ]:
nrow(impact[impact$n_depth < 20,])

In [ ]:
# [-311 rows] remove rows having n_depth < 20
impact <- impact[impact$n_depth >= 20,]

### [Remove] `t_alt_plus_count` + `t_alt_neg_count` != `t_alt_count`

In [ ]:
nrow(impact[impact$t_alt_plus_count + impact$t_alt_neg_count != impact$t_alt_count,])

In [ ]:
# [-44 rows] remove the rows having t_alt_plus_count + t_alt_neg_count != t_alt_count
impact <- impact[impact$t_alt_plus_count + impact$t_alt_neg_count == impact$t_alt_count,]

### [Create] new features: `sample_mut_key`, `patient_key`

In [ ]:
# [+1 feature] create a sample mutation key feature to idenfity unique rows
impact$sample_mut_key <- paste(impact$Tumor_Sample_Barcode, impact$mut_key, sep = '_')

In [ ]:
# [+1 feature] create a patient key feature to idenfity unique patient
impact$patient_key <- substr(impact$Tumor_Sample_Barcode, 1, 9)

### [Modify] `CDKN2Ap16INK4A` and `CDKN2Ap14ARF` reading frame

Two reading frame have been used for the gene `CDKN2A`:
* `CDKN2Ap16INK4A` is the "classic" reading frame for `CDKN2A`, the one used by OncoKB (see [OncoKB CDKN2A](http://oncokb.org/#/gene/CDKN2A))  
   → RefSeq. NM_000077.4  
   → [Homo sapiens cyclin dependent kinase inhibitor 2A (CDKN2A), transcript variant 1, mRNA](https://www.ncbi.nlm.nih.gov/nuccore/NM_000077.4)  
   → [Wikipedia article p16](https://en.wikipedia.org/wiki/P16)
* `CDKN2Ap14ARF` is an Alternative Reading Frame (ARF, as said in its name) for `CDKN2A`, not used by OncoKB  
  → RefSeq. NM_058195.3  
  → [Homo sapiens cyclin dependent kinase inhibitor 2A (CDKN2A), transcript variant 4, mRNA](https://www.ncbi.nlm.nih.gov/nuccore/NM_058195)  
  → [Wikipedia article p14arf](https://en.wikipedia.org/wiki/P14arf)
  
However, VEP thinks there's only one reading frame, the classic one `CDKN2A`. To correct this issue we:
* remove the mutations having the alternative reading frame and the classic reading frame in their tumor sample
* modify the `VEP_SYMBOL` of the mutations having the `CDKN2Ap14ARF` reading frame to `CDKN2Ap14ARF`

In [ ]:
unique(impact$VEP_SYMBOL[impact$Hugo_Symbol %in% c("CDKN2Ap16INK4A", "CDKN2Ap14ARF")])

In [ ]:
nrow(impact[impact$Hugo_Symbol == "CDKN2Ap16INK4A",])

In [ ]:
nrow(impact[impact$Hugo_Symbol == "CDKN2Ap14ARF",]) # mutations having the alternative reading data frame

dd <- impact %>% group_by(Tumor_Sample_Barcode) %>%
                 summarise(has_both_reading_frame = "CDKN2Ap14ARF" %in% Hugo_Symbol & "CDKN2Ap16INK4A" %in% Hugo_Symbol) %>%
                 filter(has_both_reading_frame)

# mutations having the alternative reading frame and the classic reading frame for this tumor sample
nrow(impact[impact$Hugo_Symbol == "CDKN2Ap14ARF" &
            impact$Tumor_Sample_Barcode %in% dd$Tumor_Sample_Barcode,]) 

We delete from `impact` the rows that are `CDKN2Ap14ARF` and have already been read in the classic reading frame:

In [ ]:
# [-713 rows] Hugo_Symbol = CDKN2Ap14ARF and CDKN2Ap16INK4A in the tumor sample
impact <- impact[! (impact$Hugo_Symbol == "CDKN2Ap14ARF" & impact$Tumor_Sample_Barcode %in% dd$Tumor_Sample_Barcode),]

### [Remove] the duplicated `sample_mut_key` rows

Some tumor sample have twice the same mutation, we keep the one with the minimal depth, and when the depth are equal the one with the minimum vaf.

In [ ]:
impact_redundant <- impact %>% group_by(sample_mut_key) %>% filter(n() >= 2)
nrow(impact_redundant)

In [ ]:
impact_redundant %>% group_by(sample_mut_key) %>% filter(n() > 3)

In [ ]:
head(impact_redundant[order(impact_redundant$sample_mut_key),], 10)

In [ ]:
impact_redundant_to_delete <- impact_redundant %>% group_by(sample_mut_key) %>% filter(t_depth == min(t_depth)) %>% filter(t_vaf == min(t_vaf))
nrow(impact_redundant_to_delete)

In [ ]:
# [-48 rows] duplicated mutation for the same sample_mut_key
impact <- impact[! (impact$sample_mut_key %in% impact_redundant_to_delete$sample_mut_key &
                    impact$t_depth %in% impact_redundant_to_delete$t_depth &
                    impact$t_vaf %in% impact_redundant_to_delete$t_vaf),]

### [None] Remove hypermutated patients?

In [ ]:
tumor_summary <- impact %>% group_by(Tumor_Sample_Barcode) %>% summarise(number_of_mutations = n())
head(tumor_summary)

In [ ]:
nrow(tumor_summary)
summary(tumor_summary$number_of_mutations)
nrow(tumor_summary[tumor_summary$number_of_mutations >= 100,])

In [ ]:
plot1 <- ggplot(tumor_summary) + geom_histogram(aes(number_of_mutations), binwidth = 1)
plot2 <- plot1 + coord_cartesian(xlim = c(100, 530), ylim = c(0, 10))
plot_side_by_side(plot1, plot2)

We decided not to remove any hypermutated patient.

### [Remove] DNP and TNP counted twice

Some `SNP` are overlapped by a `DNP` or a `TNP`, we find them and remove them.

In [ ]:
overlapping_risk_dnp_or_tnp <- as.data.frame(impact %>% group_by(Tumor_Sample_Barcode, VEP_SYMBOL) %>%
                                             filter(n() > 1 &
                                                    "SNP" %in% Variant_Type &
                                                    ("DNP" %in% Variant_Type |
                                                     "TNP" %in% Variant_Type)))

nrow(overlapping_risk_dnp_or_tnp)

In [ ]:
find_overlapping_dnp_or_tnp <- function(data, tsb, chr, start) {
    result <- data %>% filter(Tumor_Sample_Barcode == tsb &
                              Chromosome == chr &
                              ((Variant_Type == "DNP" & (Start_Position == start | Start_Position == start - 1) |
                               (Variant_Type == "TNP" & (Start_Position == start | Start_Position == start - 1 | Start_Position == start - 2)))))

    if (nrow(result) == 0)
        return ("no")
    else
        return (toString(paste(nrow(result), result$Start_Position, result$Reference_Allele, result$Tumor_Seq_Allele2, result$t_vaf, result$confidence_class, sep = ' | ')))
}

In [ ]:
overlapping_dnp_or_tnp <- overlapping_risk_dnp_or_tnp %>% filter(Variant_Type == "SNP") %>%
                                                          group_by(sample_mut_key) %>%
                                                          mutate(overlap = find_overlapping_dnp_or_tnp(overlapping_risk_dnp_or_tnp, Tumor_Sample_Barcode, Chromosome, Start_Position)) %>%
                                                          filter(overlap != "no") %>%
                                                          select(sample_mut_key, VEP_SYMBOL, Start_Position, Variant_Type, Reference_Allele, Tumor_Seq_Allele2, Tumor_Sample_Barcode, t_vaf, confidence_class, overlap)

nrow(overlapping_dnp_or_tnp)

Most of them were classified as `UNLIKELY`:

In [ ]:
get_table(overlapping_dnp_or_tnp$confidence_class)

Except for a few, the `SNP` and the overlapping `DNP/TNP` have really close `t_vaf`:

In [ ]:
overlapping_dnp_or_tnp$other_t_vaf <- sapply(overlapping_dnp_or_tnp$overlap, function(x) as.numeric(strsplit(x, ' \\| ')[[1]][5]))

In [ ]:
nrow(overlapping_dnp_or_tnp)
get_table(abs(overlapping_dnp_or_tnp$t_vaf - overlapping_dnp_or_tnp$other_t_vaf) > 0.01)
get_table(abs(overlapping_dnp_or_tnp$t_vaf - overlapping_dnp_or_tnp$other_t_vaf) > 0.05)

In [ ]:
# [-3151 rows] SNV found as DNP or TNP
impact <- impact[! impact$sample_mut_key %in% overlapping_dnp_or_tnp$sample_mut_key,]

## Process raw features

### [Modify] deal with `NA` values

In [ ]:
count_na <- function(data) {
    return (sum(is.na(data)))
}

In [ ]:
replace_na <- function(data, feature_name, replace_value){
    data[is.na(data[,feature_name]), feature_name] <- replace_value
    
    return (data)
}

#### VAG

In [ ]:
for (c in vag_colnames)
    print(sprintf("%17s: %d", c, count_na(impact[,c])))

In [ ]:
impact <- replace_na(impact, "VAG_GENE"          , "unknown")
impact <- replace_na(impact, "VAG_cDNA_CHANGE"   , "unknown")
impact <- replace_na(impact, "VAG_PROTEIN_CHANGE", "unknown")
impact <- replace_na(impact, "VAG_EFFECT"        , "unknown")

#### VEP

In [ ]:
for (c in vep_colnames)
    print(sprintf("%17s: %d", c, count_na(impact[,c])))

In [ ]:
impact[is.na(impact$VEP_HGVSc),]

7280 mutations have `VEP_HGVSp` = `NA`, almost all of them being splicing mutations:

In [ ]:
get_table(impact$VEP_Consequence[is.na(impact$VEP_HGVSp)])

7280 mutations have `VEP_Amino_acids` = `NA`, most of them being the one having `VEP_HGVSp = NA`:

In [ ]:
count_na(impact$VEP_Amino_acids)
table(impact$mut_key[is.na(impact$VEP_Amino_acids)] %in% impact$mut_key[is.na(impact$VEP_HGVSp)])

In [ ]:
get_table(impact$VEP_Consequence[is.na(impact$VEP_Amino_acids) & ! is.na(impact$VEP_HGVSp)])

In [ ]:
nrow(impact[impact$VEP_Consequence == "splice_acceptor_variant" & ! is.na(impact$VEP_HGVSp) & impact$VEP_HGVSp != "unknown",])
nrow(impact[impact$VEP_Consequence == "splice_donor_variant" & ! is.na(impact$VEP_HGVSp) & impact$VEP_HGVSp != "unknown",])

In [ ]:
impact <- replace_na(impact, "VEP_HGVSc"      , "unknown")
impact <- replace_na(impact, "VEP_HGVSp"      , "unknown")
impact <- replace_na(impact, "VEP_Amino_acids", "unknown")

`VEP_EXON` and `VEP_INTRON` are complementary: when one is `NA` the other has a value, except for 41 cases.

In [ ]:
get_table(is.na(impact$VEP_EXON) & is.na(impact$VEP_INTRON))

#### VEP additional

In [ ]:
for (c in vep_add_colnames)
    print(sprintf("%22s: %d", c, count_na(impact[,c])))

`NA` values might correspond to not found, we replace them by `"unknown"`:

In [ ]:
impact <- replace_na(impact, "VEP_CLIN_SIG"          , "unknown")
impact <- replace_na(impact, "VEP_SIFT"              , "unknown")
impact <- replace_na(impact, "VEP_PolyPhen"          , "unknown")
impact <- replace_na(impact, "VEP_Existing_variation", "unknown")
impact <- replace_na(impact, "VEP_COSMIC_CNT"        , "unknown")

#### VEP gnomAD

In [ ]:
for (c in vep_gnomad_colnames)
    print(sprintf("%27s: %d", c, count_na(impact[,c])))

`NA` values might correspond to not found, we replace them by a null allele value, or by `0 | 0` for the Allele count | Total number of alleles:

In [ ]:
impact <- replace_na(impact, "VEP_gnomAD_AF", 0.0)

for (c in vep_gnomad_colnames[grepl("_AC.AN_", vep_gnomad_colnames)])
    impact <- replace_na(impact, c, " 0 | 0")

### [Modify] `occurence_in_normals` -> `frequency_in_normals`

In [ ]:
dd <- head(unique(impact$occurence_in_normals), 10)
dd[dd == '0'] <- "0;0"
dd <- data.frame(dd, sapply(dd, function(s) as.double(strsplit(s, split = ';')[[1]][2])), row.names = NULL)
colnames(dd) <- c("old", "new")
dd

In [ ]:
# [~ every rows] occurence_in_normals -> frequency_in_normals
impact$occurence_in_normals[impact$occurence_in_normals == '0'] <- "0;0"
impact$frequency_in_normals <- sapply(impact$occurence_in_normals,
                                      function(s) as.double(strsplit(s, split = ';')[[1]][2]))
impact$occurence_in_normals <- NULL

### [Modify] `VEP_HGVSc`

In [ ]:
dd <- head(impact$VEP_HGVSc, 10)
dd <- data.frame(dd, sapply(dd, function(x) strsplit(x, ':')[[1]][2]), row.names = NULL)
colnames(dd) <- c("old", "new")
dd

In [ ]:
# [~ every rows] VEP_HGVSc -> readable VEP_HGVSc
impact$VEP_HGVSc <- sapply(impact$VEP_HGVSc, function(x) strsplit(x, ':')[[1]][2])
impact <- replace_na(impact, "VEP_HGVSc", "unknown") # 5 NA values that we need to handle

### [Modify] `VEP_HGVSp`

In [ ]:
get_HGVSp_from_vep <- function(HGVSp_string) {
    
    if (HGVSp_string == "unknown")
        return ("unknown")
    
    HGVSp_string <- strsplit(HGVSp_string, ':')[[1]][2]
    
    protein_long_name <- c('Ala', 'Arg', 'Asn', 'Asp', 'Cys', 'Glu', 'Gln', 'Gly', 'His', 'Ile', 'Leu', 'Lys',
                           'Met', 'Phe', 'Pro', 'Ser', 'Thr', 'Trp', 'Tyr', 'Val')
    protein_short_name <- c('A', 'R', 'N', 'D', 'C', 'E', 'Q', 'G', 'H', 'I', 'L', 'K',
                            'M', 'F', 'P', 'S', 'T', 'W', 'Y', 'V')
    
    for (name in protein_long_name)
        HGVSp_string <- gsub(name, protein_short_name[match(name, protein_long_name)], HGVSp_string)
    
    HGVSp_string <- gsub('Ter', '*', HGVSp_string)
    HGVSp_string <- gsub('%3D', '=', HGVSp_string)
    
    return (HGVSp_string)
}

In [ ]:
dd <- head(impact$VEP_HGVSp, 10)
dd <- data.frame(dd, sapply(dd, get_HGVSp_from_vep), row.names = NULL)
colnames(dd) <- c("old", "new")
dd

In [ ]:
# [~ every rows] VEP_HGVSp -> readable VEP_HGVSp
impact$VEP_HGVSp <- sapply(impact$VEP_HGVSp, get_HGVSp_from_vep)

### [Modify] `VEP_SIFT`

In [ ]:
dd <- head(unique(impact$VEP_SIFT), 10)
dd <- data.frame(dd,
                 sapply(dd, function(x) strsplit(x, '\\(')[[1]][1]),
                 sapply(dd, function(x) as.numeric(gsub(')', '', strsplit(x, '\\(')[[1]][2]))),
                 row.names = NULL)
colnames(dd) <- c("old", "new1", "new2")
dd

In [ ]:
# [~ every rows] VEP_SIFT -> VEP_SIFT_class & VEP_SIFT_score
impact$VEP_SIFT_class <- sapply(impact$VEP_SIFT, function(x) strsplit(x, '\\(')[[1]][1])
impact$VEP_SIFT_score <- sapply(impact$VEP_SIFT, function(x) as.numeric(gsub(')', '', strsplit(x, '\\(')[[1]][2])))
impact$VEP_SIFT <- NULL

Warning, there is `NA` values in `VEP_SIFT_score` for `unknown`.

### [Modify] `VEP_PolyPhen`

In [ ]:
dd <- head(unique(impact$VEP_PolyPhen), 10)
dd <- data.frame(dd,
                 sapply(dd, function(x) strsplit(x, '\\(')[[1]][1]),
                 sapply(dd, function(x) as.numeric(gsub(')', '', strsplit(x, '\\(')[[1]][2]))),
                 row.names = NULL)
colnames(dd) <- c("old", "new1", "new2")
dd

In [ ]:
# [~ every rows] VEP_PolyPhen -> VEP_PolyPhen_class & VEP_PolyPhen_score
impact$VEP_PolyPhen_class <- sapply(impact$VEP_PolyPhen, function(x) strsplit(x, '\\(')[[1]][1])
impact$VEP_PolyPhen_score <- sapply(impact$VEP_PolyPhen, function(x) as.numeric(gsub(')', '', strsplit(x, '\\(')[[1]][2])))
impact$VEP_PolyPhen <- NULL

Warning, there is `NA` values in `VEP_PolyPhen_score` for `unknown`.

### [Modify] `VEP_Existing_variation` -> `VEP_in_dbSNP`

A mutation has the feature`VEP_in_dbSNP = TRUE` when we found a dnSNP id in the `VEP_Existing_variation` column.

In [ ]:
dd <- head(unique(impact$VEP_Existing_variation), 10)
dd <- data.frame(dd, grepl("rs", dd), row.names = NULL)
colnames(dd) <- c("old", "new")
dd

In [ ]:
# [~ every rows] VEP_Existing_variation -> VEP_in_dbSNP
impact$VEP_in_dbSNP <- grepl("rs", impact$VEP_Existing_variation)
impact$VEP_Existing_variation <- NULL

In [ ]:
get_table(impact$VEP_in_dbSNP)

In [ ]:
plot_contingency_table_as_histograms(impact, "confidence_class", "VEP_in_dbSNP", height = 2.5)

We clearly see the germline `t_vaf` for the `UNLIKELY` class.

In [ ]:
plot_density(impact, "t_vaf", fill = "VEP_in_dbSNP", height = 2.5) + facet_grid(col = vars(confidence_class))

### [Modify] `VEP_COSMIC_CNT`

In [ ]:
get_cosmic_count_from_vep <- function(cosmic_count_string) {
    if (cosmic_count_string == "unknown")
        return (0)
    else
        return (sum(as.numeric(strsplit(cosmic_count_string, '&')[[1]])))
}

dd <- head(unique(impact$VEP_COSMIC_CNT), 10)
dd <- data.frame(dd, sapply(dd, get_cosmic_count_from_vep), row.names = NULL)
colnames(dd) <- c("old", "new")
dd

In [ ]:
# [~ every rows] VEP_COSMIC_CNT -> readable VEP_COSMIC_CNT
impact$VEP_COSMIC_CNT <- sapply(impact$VEP_COSMIC_CNT, get_cosmic_count_from_vep)

### [Modify] VEP_CLIN_SIG

In [ ]:
get_table(impact$VEP_CLIN_SIG) %>% filter(count >= 200)

We group the values with the following rules:
* We consider 3 big different classes:
    * `pathogenic`: `pathogenic`, `likely_pathogenic`, `drug_response`, `risk_factor`
    * `benign`: `benign`, `likely_benign`
    * `unknown`: `NA` (has been previously replaced by `unknown`), `not_provided`, `uncertain_significance`
* A mutation having both `pathogenic` and `benign` classes is classified as `unknown`
* A mutation having a strong class (`pathogenic` or `benign`) and `unknown` class is classified as the strong class

In [ ]:
get_simplified_clin_sig <- function(clin_sig_string) {
    if (clin_sig_string == "unknown")
        return ("unknown")
    else {
        tags <- unique(strsplit(clin_sig_string, '&')[[1]])
        
        tags <- gsub('likely_pathogenic', 'pathogenic', tags)
        tags <- gsub('drug_response'    , 'pathogenic', tags)
        tags <- gsub('risk_factor'      , 'pathogenic', tags)
        tags <- gsub('likely_benign'    , 'benign'    , tags)
        
        tags <- unique(tags)
        
        tags <- tags[! tags %in% c("not_provided", "uncertain_significance", "other")]
        
        if (length(tags) == 0 || length(tags) > 1)
            return ("unknown")
        else
            return (tags)
    }
}

In [ ]:
dd <- head(unique(impact$VEP_CLIN_SIG), 20)
dd <- data.frame(dd, sapply(dd, get_simplified_clin_sig), row.names = NULL)
colnames(dd) <- c("old", "new")
dd

In [ ]:
# [~ every rows] VEP_CLIN_SIG -> readable VEP_CLIN_SIG
impact$VEP_CLIN_SIG <- sapply(impact$VEP_CLIN_SIG, get_simplified_clin_sig)

In [ ]:
get_table(impact$VEP_CLIN_SIG)

### [Modify & Create & Remove] vep_gnomad_colnames

In [ ]:
pop_names <- c('AFR', 'AMR', 'ASJ', 'EAS', 'FIN', 'NFE', 'OTH')

#### `genome_AC.AN_<POP>` & `exome_AC.AN_<POP>` -> `total_AC.AN_<POP>`

For each population, we sum the allele count (`AC`) and total allele number (`AN`) on the exomes and the genomes. This operation creates 7 new features named `VEP_gnomAD_total_AC.AN_<POP>`.

In [ ]:
get_gnomAD_total_AC.AN_pop <- function(data, pop_name) {
    genome_AC = as.integer(strsplit(data[paste0("VEP_gnomAD_genome_AC.AN_", pop_name)], ' \\| ')[[1]][1])
    genome_AN = as.integer(strsplit(data[paste0("VEP_gnomAD_genome_AC.AN_", pop_name)], ' \\| ')[[1]][2])

    exome_AC = as.integer(strsplit(data[paste0("VEP_gnomAD_exome_AC.AN_", pop_name)], ' \\| ')[[1]][1])
    exome_AN = as.integer(strsplit(data[paste0("VEP_gnomAD_exome_AC.AN_", pop_name)], ' \\| ')[[1]][2])
    
    return (paste(genome_AC + exome_AC, genome_AN + exome_AN, sep = ' | '))
}

In [ ]:
dd <- tail(unique(impact[, c("VEP_gnomAD_genome_AC.AN_AFR", "VEP_gnomAD_exome_AC.AN_AFR")]), 10)
dd <- data.frame(dd, apply(dd, 1, function(x) get_gnomAD_total_AC.AN_pop(x, "AFR")), row.names = NULL)
colnames(dd) <- c("genome_AC.AN_AFR", "exome_AC.AN_AFR", "new (total_AC.AN_AFR)")
dd

In [ ]:
# [+7 features] VEP_gnomAD_total_AC.AN_<POP> (temporary feature)
for (pop in pop_names)
    impact[, paste0("VEP_gnomAD_total_AC.AN_", pop)] <- apply(impact, 1, function(x) get_gnomAD_total_AC.AN_pop(x, pop))

#### `total_AC.AN_<POP>` -> `total_AF_<POP>`

Foe each population, we convert `VEP_gnomAD_total_AC.AN_<POP>` to the corresponding allele frequency (`AF`), the allele frequency of the mutation in gnomAD exomes and genomes united. This operation creates 7 new features named `VEP_gnomAD_total_AF_<POP>`.

In [ ]:
get_gnomAD_total_AF_pop <- function(data, pop_name) {
        
    AC = as.integer(strsplit(data[paste0("VEP_gnomAD_total_AC.AN_", pop_name)], ' \\| ')[[1]][1])
    AN = as.integer(strsplit(data[paste0("VEP_gnomAD_total_AC.AN_", pop_name)], ' \\| ')[[1]][2])
    
    if (AN == 0)
        return (0)
    else
        return (AC / AN)
}

In [ ]:
dd <- tail(unique(impact[, c("VEP_gnomAD_total_AC.AN_AFR",
                             "VEP_gnomAD_total_AC.AN_AMR")]), 10)
dd <- data.frame(dd, apply(dd, 1, function(x) get_gnomAD_total_AF_pop(x, "AFR")),
                     apply(dd, 1, function(x) get_gnomAD_total_AF_pop(x, "AMR")), row.names = NULL)
colnames(dd) <- c("total_AC.AN_AFR", "total_AC.AN_AMR", "new1 (total_AF_AFR)", "new2 (total_AF_AMR)")
dd

In [ ]:
# [+7 features] VEP_gnomAD_total_AF_<POP>
for (pop in pop_names)
    impact[, paste0("VEP_gnomAD_total_AF_", pop)] <- apply(impact, 1, function(x) get_gnomAD_total_AF_pop(x, pop))

#### `total_AC.AN_<POP>` -> `VEP_gnomAD_total_AF_max`

We calculate the maximal `AF` among all 7 populations, for gnomAD exomes and genomes united. This operation creates 1 new feature named `VEP_gnomAD_total_AF_max`.

In [ ]:
dd <- tail(unique(impact[, c("VEP_gnomAD_total_AF_AFR",
                             "VEP_gnomAD_total_AF_AMR")]), 10)

total_AF_columns <- colnames(dd)[grepl("VEP_gnomAD_total_AF_", colnames(dd))]

dd <- data.frame(dd, apply(dd, 1, function(x) max(as.numeric(x[total_AF_columns]))), row.names = NULL)
colnames(dd) <- c("total_AF_AFR", "total_AF_AMR", "new")
dd

In [ ]:
# [+1 feature] VEP_gnomAD_total_AF_max
total_AF_columns <- colnames(impact)[grepl("VEP_gnomAD_total_AF_", colnames(impact))]
impact$VEP_gnomAD_total_AF_max <- apply(impact, 1, function(x) max(as.numeric(x[total_AF_columns])))

#### `total_AC.AN_<POP>` -> `VEP_gnomAD_total_AF`

We calculate the `AF` among all 7 populations, for gnomAD exomes and genomes united. This operation creates 1 new feature named `VEP_gnomAD_total_AF`.

In [ ]:
get_gnomAD_total_AF <- function(data) {
    
    AC <- c()
    AN <- c()
    
    for (pop in pop_names) {
        AC <- c(AC, as.integer(strsplit(data[paste0("VEP_gnomAD_total_AC.AN_", pop)], ' \\| ')[[1]][1]))
        AN <- c(AN, as.integer(strsplit(data[paste0("VEP_gnomAD_total_AC.AN_", pop)], ' \\| ')[[1]][2]))
    }
                
    if (sum(AN) == 0)
        return (0)
    else
        return (sum(AC) / sum(AN))
}

In [ ]:
total_AC.AN_columns <- colnames(impact)[grepl("VEP_gnomAD_total_AC.AN_", colnames(impact))]

dd <- tail(unique(impact[, total_AC.AN_columns]), 10)
dd <- data.frame(dd, apply(dd, 1, get_gnomAD_total_AF), row.names = NULL)
colnames(dd) <- c(total_AC.AN_columns, "new")
dd

In [ ]:
# [+1 feature] VEP_gnomAD_total_AF
impact$VEP_gnomAD_total_AF <- apply(impact, 1, get_gnomAD_total_AF)

#### Results

Here is a comparison plot between `VEP_gnomAD_AF` vs `VEP_gnomAD_total_AF` on the left, and `VEP_gnomAD_total_AF` vs `VEP_gnomAD_total_AF_max` on the right:

In [ ]:
plot1 <- plot_density_2d(impact, "VEP_gnomAD_AF", "VEP_gnomAD_total_AF", points_only = TRUE, height = 2) + geom_abline(intercept = 0, slope = 1, color ="purple", size = 0.6)
plot2 <- plot_density_2d(impact, "VEP_gnomAD_total_AF", "VEP_gnomAD_total_AF_max", points_only = TRUE, height = 2) + geom_abline(intercept = 0, slope = 1, color ="purple", size = 0.6)
plot_side_by_side(plot1, plot2)

We remove the features we won't use, ie:
* `VEP_gnomAD_genome_AC.AN_<POP>`: 7 features
* `VEP_gnomAD_exome_AC.AN_<POP>`: 7 features
* `VEP_gnomAD_total_AC.AN_<POP>`: 7 features

In [ ]:
# [-21 features] remove VEP_gnomAD_genome_AC.AN_<POP>, VEP_gnomAD_exome_AC.AN_<POP> and VEP_gnomAD_total_AC.AN_<POP>
impact[, colnames(impact)[grepl("VEP_gnomAD_genome_AC.AN", colnames(impact))]] <- NULL
impact[, colnames(impact)[grepl("VEP_gnomAD_exome_AC.AN", colnames(impact))]] <- NULL
impact[, colnames(impact)[grepl("VEP_gnomAD_total_AC.AN", colnames(impact))]] <- NULL

In [ ]:
vep_gnomad_colnames <- c("VEP_gnomAD_AF",
                         "VEP_gnomAD_total_AF_AFR",
                         "VEP_gnomAD_total_AF_AMR",
                         "VEP_gnomAD_total_AF_ASJ",
                         "VEP_gnomAD_total_AF_EAS",
                         "VEP_gnomAD_total_AF_FIN",
                         "VEP_gnomAD_total_AF_NFE",
                         "VEP_gnomAD_total_AF_OTH",
                         "VEP_gnomAD_total_AF_max",
                         "VEP_gnomAD_total_AF")